In [0]:
!pip install visdom
!pip install tensorboardX 
!pip install tensorflow
!pip install image_slicer

In [0]:
from google.colab import drive
drive.mount('/content/drive')
root = '/content/drive/My Drive/Colab Notebooks/Deep Learning/EndoVis2018/code'
%cd /content/drive/My\ Drive/Colab\ Notebooks/Deep\ Learning/EndoVis2018/code

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/Deep Learning/EndoVis2018/code


In [0]:
# This code is used for JHU CS 482/682: Deep Learning 2019 Spring Project
# Copyright: Zhaoshuo Li, Ding Hao, Mingyi Zheng
import os

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as functional
from torch.utils.data import DataLoader, Dataset
# from torchvision import transforms
import torchvision.transforms.functional as TF

import copy
import numpy as np
import random
from tensorboardX import SummaryWriter

import transforms
from dataset import *
from visualization import *
from label_conversion import *
from dice_loss import *
from model_trainning import *

#from unet import *
from deeplabv3p_xception import *

In [0]:
#os.environ["CUDA_VISIBLE_DEVICES"]="2"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cuda


# Seed pytorch and numpy and random

In [0]:
# IMPORTANT!
# must seed the same value each time when training a new network
seed = 256
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
pretrain_seed = 128

## Hyperparameters

In [0]:
train_batch_size = 16
validation_batch_size=16
learning_rate = 0.001
num_epochs = 70
num_class = 12

In [0]:
dice_loss = DICELoss(np.ones((num_class,1))) 

## Visualization

In [0]:
# Initialize the visualization environment
writer = SummaryWriter()

##Unet

In [0]:
# initialize model
model = unet(useBN=True)
model.to(device)

## Deeplab v3+ with Xception

In [0]:
# initialize model
model = DeepLabv3_plus(n_classes=12)
model.to(device)

## Optimizer and Scheduler and loss

In [0]:
# intialize optimizer and lr decay
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

## Baseline, without augmentation

In [0]:
# IMPORTANT!
# must seed the same value each time when training a new network
seed = 256
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

In [0]:
# define transform
train_dataset=MICCAIDataset(data_type = "train", transform=None)
validation_dataset=MICCAIDataset(data_type = "validation", transform=None)
label_converter = LabelConverter()

# # show one example
# img,label = train_dataset.__getitem__(0)
# imshow(img.permute(1,2,0),denormalize=True)
# colorlabel = train_dataset.label_converter.label2color(label.permute(1,2,0))
# imshow(colorlabel)

# # show one example
# img,label = validation_dataset.__getitem__(0)
# imshow(img.permute(1,2,0),denormalize=True)
# colorlabel = train_dataset.label_converter.label2color(label.permute(1,2,0))
# imshow(colorlabel)

In [0]:
# intialize the dataloader
train_generator = DataLoader(train_dataset,shuffle=True,batch_size=train_batch_size,num_workers=8)
validation_generator = DataLoader(validation_dataset,shuffle=True,batch_size=validation_batch_size,num_workers=8)

## Start training

In [0]:
print("Training Started!")

# initialize best_acc for comparison
best_acc = 0.0
train_iter = 0
val_iter = 0

for epoch in range(num_epochs):
    print("\nEPOCH " +str(epoch+1)+" of "+str(num_epochs)+"\n")
    
    # train
    train_loss, train_iter = train(model,device,scheduler,optimizer,dice_loss,train_generator,train_dataset,writer,train_iter)
    print(train_iter)

    # validate
    with torch.no_grad():
        validation_loss, tp, fp, fn, val_iter = validate(model,device,dice_loss,num_class,validation_generator,validation_dataset,writer,val_iter)
        epoch_acc = (2*tp + 1e-7)/ (2*tp+fp+fn+1e-7)
        epoch_acc = epoch_acc.mean()
    
        # loss
        writer.add_scalar('data/Training Loss (per epoch)',train_loss,epoch)
        writer.add_scalar('data/Validation Loss (per epoch)',validation_loss,epoch)
        
        # randomly show one validation image 
        sample = validation_dataset.__getitem__(random.randint(0,len(validation_dataset)-1))
        img = sample[0]*0.5+0.5
        label = sample[1]
        tmp_img = sample[0].reshape(1,3,256,320)
        pred = functional.softmax(model(tmp_img.cuda()), dim=1)
        pred_label = torch.max(pred,dim=1)[1]
        pred_label = pred_label.type(label.type())
        # to plot
        tp_img = np.array(img)
        tp_label = train_dataset.label_converter.label2color(label.permute(1,2,0)).transpose(2,0,1)
        tp_pred = train_dataset.label_converter.label2color(pred_label.permute(1,2,0)).transpose(2,0,1)
        
        writer.add_image('Input', tp_img, epoch)
        writer.add_image('Label', tp_label, epoch)
        writer.add_image('Prediction', tp_pred, epoch)
        
        # deep copy the model
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

Training Started!

EPOCH 1 of 70



/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2457: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Epoch Loss: 0.7462
----------
90
Vaildation Loss: 0.6639
0 Class, True Pos 9718419.0, False Pos 3001950.0, Flase Neg 3542839.0
1 Class, True Pos 1783534.0, False Pos 829719.0, Flase Neg 733393.0
2 Class, True Pos 857344.0, False Pos 941929.0, Flase Neg 449622.0
3 Class, True Pos 248363.0, False Pos 415264.0, Flase Neg 304974.0
4 Class, True Pos 3641530.0, False Pos 2232248.0, Flase Neg 1261133.0
5 Class, True Pos 2007350.0, False Pos 551108.0, Flase Neg 1460697.0
6 Class, True Pos 5742.0, False Pos 59323.0, Flase Neg 80675.0
7 Class, True Pos 12341.0, False Pos 38218.0, Flase Neg 122747.0
8 Class, True Pos 0.0, False Pos 0.0, Flase Neg 453.0
9 Class, True Pos 0.0, False Pos 0.0, Flase Neg 112924.0
10 Class, True Pos 2054431.0, False Pos 846627.0, Flase Neg 672957.0
11 Class, True Pos 0.0, False Pos 0.0, Flase Neg 173972.0
----------

EPOCH 2 of 70

Epoch Loss: 0.6094
----------
180
Vaildation Loss: 0.6014
0 Class, True Pos 9970730.0, False Pos 2580595.0, Flase Neg 3290528.0
1 Class, Tr

In [0]:
## load best model weights
model.load_state_dict(best_model_wts)
## save model
torch.save(model.state_dict(), 'trained_deeplabv3plus.pt')

## Test

In [0]:
# load test dataset
test_dataset=MICCAIDataset(data_type = "test", transform=None)
test_generator=DataLoader(test_dataset,shuffle=False,batch_size=4,num_workers=8)

In [0]:
# load model
model.load_state_dict(torch.load('trained_deeplabv3plus.pt'))
model.to(device)
print("Model loaded")

Model loaded


In [0]:
final_dice = test(model,device,dice_loss,num_class,test_generator,test_dataset,writer)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2457: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Dice Score: 0.7690
0 Class, True Pos 15719158.0, False Pos 1095669.0, Flase Neg 1175737.0
1 Class, True Pos 2907797.0, False Pos 317131.0, Flase Neg 220393.0
2 Class, True Pos 1469173.0, False Pos 315624.0, Flase Neg 333436.0
3 Class, True Pos 672369.0, False Pos 166023.0, Flase Neg 181162.0
4 Class, True Pos 6018225.0, False Pos 533037.0, Flase Neg 450104.0
5 Class, True Pos 3841554.0, False Pos 298929.0, Flase Neg 340552.0
6 Class, True Pos 78306.0, False Pos 43167.0, Flase Neg 42559.0
7 Class, True Pos 122913.0, False Pos 30151.0, Flase Neg 34316.0
8 Class, True Pos 0.0, False Pos 0.0, Flase Neg 287.0
9 Class, True Pos 85197.0, False Pos 17157.0, Flase Neg 64858.0
10 Class, True Pos 2495322.0, False Pos 159868.0, Flase Neg 150195.0
11 Class, True Pos 192473.0, False Pos 38997.0, Flase Neg 22154.0
----------


# Data Augmentation

# Pretraining